In [10]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import socket
import pandas as pd
import pdfplumber
import os
from tqdm import tqdm
import xlrd
import xlwt

In [16]:
# Load the combined search output CSV file
df = pd.read_csv('Investment Research/Investment_Research_All_V2.csv')

# Convert the date column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y')

# Extract the month, day, and year from the date column
df['Month'] = df['Date'].dt.strftime('%b')
df['Day'] = df['Date'].dt.day
df['Year'] = df['Date'].dt.year

# Create the new Join_Key column
df['Join_Key'] = df['Headline2'] + '_' + df['Month'] + '_' + df['Day'].astype(str) + ',_' + df['Year'].astype(str)

# Display the resulting DataFrame
print(df.head())

# Save the DataFrame to a new CSV file
df.to_csv('Investment Research/Investment_Research_All_V2_Formatted.csv', index=False)

            Contributor              Analyst      Date/Time Company  \
0  CFRA Equity Research  Handshoe, Jonnathan  5/16/24 22:10     MRO   
1  CFRA Equity Research    Glickman, Stewart  5/14/24 22:02     EOG   
2  CFRA Equity Research    Glickman, Stewart  5/11/24 18:08     EOG   
3  CFRA Equity Research    Glickman, Stewart  5/11/24 17:57     DVN   
4  CFRA Equity Research    Glickman, Stewart   5/7/24 22:03     COP   

                   Headline  Pages       Date               Headline2 Month  \
0  Marathon Oil Corporation      9 2024-05-16  MarathonOilCorporation   May   
1       EOG Resources, Inc.      9 2024-05-14       EOGResources,Inc.   May   
2       EOG Resources, Inc.      9 2024-05-11       EOGResources,Inc.   May   
3  Devon Energy Corporation      9 2024-05-11  DevonEnergyCorporation   May   
4            ConocoPhillips      9 2024-05-07          ConocoPhillips   May   

   Day  Year                             Join_Key  
0   16  2024  MarathonOilCorporation_May_16,_2

In [17]:
def extract_text_and_title(pdf_path):
    try:
        with pdfplumber.open(pdf_path) as pdf:
            first_page = pdf.pages[0]
            text = first_page.extract_text()
        return text
    except Exception as e:
        print("Error reading " + pdf_path + ": " + str(e))
        return None

def extract_date_from_filename(filename):
    # Regex to match the date format in the filename
    date_pattern = r'(\w{3})_(\d{1,2}),_(\d{4})\.pdf'
    match = re.search(date_pattern, filename)
    if match:
        # Construct date string and return it
        return match.group(1) + " " + match.group(2) + ", " + match.group(3)
    else:
        return 'Date not found'
    
def extract_document_id(title):
    # Regex to match the document ID pattern
    doc_id_pattern = r'(\d+)[^_]*'
    match = re.search(doc_id_pattern, title)
    if match:
        return match.group(1)
    else:
        return 'ID not found'
    
def extract_join_key(filename):
    # Extract the part of the filename following the first _ and ending just prior to .pdf
    join_key_pattern = r'_(.*?)\.pdf'
    match = re.search(join_key_pattern, filename)
    if match:
        return match.group(1)
    else:
        return 'Join_Key not found'

# Directory containing PDFs
pdf_dir = 'Investment Research/PDFs_All_V2'
pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith('.pdf')]

# Extract data from each PDF
data = []
for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_dir, pdf_file)
    text = extract_text_and_title(pdf_path)
    if text is None:
        continue  # Skip files that couldn't be read
    title = pdf_file  # Use file name as title
    source = pdf_file.split('_')[0]  # Extract source from file name
    date = extract_date_from_filename(pdf_file)  # Extract date from filename
    unique_id = extract_document_id(title)
    join_key = extract_join_key(pdf_file)  # Extract join key from filename
    data.append({'Unique_ID': unique_id,'Join_Key': join_key, 'Date': date, 'Title': title, 'Source': source, 'Text': text})

# Create a DataFrame
df = pd.DataFrame(data)

# Ensure Join_Key is a string and clean it
def clean_join_key(key):
    key = re.sub(r'[^\w\s]', '', key.lower())
    key = re.sub(r'_(0)', '_', key)  # Remove '0' following the second underscore
    return key

df['Join_Key'] = df['Join_Key'].astype(str).apply(clean_join_key)

# Save to CSV for new reference file
csv_path = 'Investment Research/Investment_Research_All_DF.csv'
df.to_csv(csv_path, index=False)
print('Data saved to ' + csv_path)

# Load existing CSV from search .csv
existing_csv_path = 'Investment Research/Investment_Research_All_V2_Formatted.csv'
existing_df = pd.read_csv(existing_csv_path)

# Ensure Join_Key is a string and clean it in the existing DataFrame
existing_df['Join_Key'] = existing_df['Join_Key'].astype(str).apply(clean_join_key)

# Merge the 'Text' column from new data to the existing DataFrame based on 'Join_Key'
merged_df = existing_df.merge(df[['Join_Key', 'Text']], on='Join_Key', how='left')

# Select only the desired columns
columns_to_keep = ['Contributor', 'Date/Time', 'Date', 'Company', 'Headline', 'Text']
final_df = merged_df[columns_to_keep]

# Replace values in the "Company" column
replacements = {
    "Pioneer Natural Resources Company": "PXD",
    "Concho Resources inc.": "CXO",
    "BP.": "BP",
    "PDC Energy, Inc.": "PDCE"
}

final_df['Company'] = final_df['Company'].replace(replacements)

# Rename the "Company" column to "Ticker"
final_df.rename(columns={"Company": "Ticker"}, inplace=True)

# Save to CSV
csv_path2 = 'Investment Research/Investment_Research_ALLV2_Final.csv'
final_df.to_csv(csv_path2, index=False)
print('Updated data saved to ' + csv_path2)

Error reading Investment Research/PDFs_All_V2/MarathonOilCorporation_MarathonOilCorporation_Feb_15,_2020.pdf: list index out of range
Error reading Investment Research/PDFs_All_V2/ConocoPhillips_ConocoPhillipsCompany_Feb_22,_2020.pdf: Unexpected EOF
Error reading Investment Research/PDFs_All_V2/PioneerNaturalResourcesCompany_PioneerNaturalResourcesCompany_Feb_22,_2020.pdf: Unexpected EOF
Error reading Investment Research/PDFs_All_V2/EOGResources,Inc_EOGResources,Inc_Feb_09,_2020.pdf: Unexpected EOF
Error reading Investment Research/PDFs_All_V2/ValeroEnergyCorporation_ValeroEnergyCorporation_Feb_22,_2020.pdf: Unexpected EOF
Data saved to Investment Research/Investment_Research_All_DF.csv
Updated data saved to Investment Research/Investment_Research_ALLV2_Final.csv


In [31]:
df

,Unique_ID,Join_Key,Date,Title,Source,Text
0,02,occidentalpetroleumcorporation_may_02_2020,"May 02, 2020",OccidentalPetroleumCorporation_OccidentalPetro...,OccidentalPetroleumCorporation,"Stock Report | May 02, 2020 | NYSE Symbol: OXY..."
1,28,chevroncorporation_aug_28_2021,"Aug 28, 2021","ChevronCorporation_ChevronCorporation_Aug_28,_...",ChevronCorporation,"Stock Report | August 28, 2021 | NYSE Symbol: ..."
2,28,bpplc_oct_28_2023,"Oct 28, 2023","BPplc_BPplc_Oct_28,_2023.pdf",BPplc,"Stock Report | October 28, 2023 | NYSE Symbol:..."
3,25,marathonpetroleumcorporation_jan_25_2020,"Jan 25, 2020",MarathonPetroleumCorporation_MarathonPetroleum...,MarathonPetroleumCorporation,"Stock Report | January 25, 2020 | NYSE Symbol:..."
4,02,chevroncorporation_nov_02_2020,"Nov 02, 2020","ChevronCorporation_ChevronCorporation_Nov_02,_...",ChevronCorporation,"Stock Report | November 02, 2020 | NYSE Symbol..."
...,...,...,...,...,...,...
4835,05,shellplc_aug_05_2023,"Aug 05, 2023","Shellplc_Shellplc_Aug_05,_2023.pdf",Shellplc,"Stock Report | August 05, 2023 | NYSE Symbol: ..."
4836,27,exxonmobilcorporation_feb_27_2021,"Feb 27, 2021",ExxonMobilCorporation_ExxonMobilCorporation_Fe...,ExxonMobilCorporation,"Stock Report | February 27, 2021 | NYSE Symbol..."
4837,09,marathonoilcorporation_jan_09_2022,"Jan 09, 2022",MarathonOilCorporation_MarathonOilCorporation_...,MarathonOilCorporation,"Stock Report | January 08, 2022 | NYSE Symbol:..."
4838,22,energytransitionclimatealignmentaproxyfortrans...,"Apr 22, 2024","BPplc_EnergyTransition-ClimateAlignment,aproxy...",BPplc,PARIS\nMALIGNED\nII\nClimate alignment assessm...


In [21]:
merged_df

,Contributor,Analyst,Date/Time,Company,Headline,Pages,Date,Headline2,Month,Day,Year,Join_Key,Text
0,CFRA Equity Research,"Handshoe, Jonnathan",5/16/24 22:10,MRO,Marathon Oil Corporation,9,2024-05-16,MarathonOilCorporation,May,16,2024,"MarathonOilCorporation_May_16,_2024","Stock Report | May 16, 2024 | NYSESymbol: MRO ..."
1,CFRA Equity Research,"Glickman, Stewart",5/14/24 22:02,EOG,"EOG Resources, Inc.",9,2024-05-14,"EOGResources,Inc.",May,14,2024,"EOGResources,Inc._May_14,_2024",NaN
2,CFRA Equity Research,"Glickman, Stewart",5/11/24 18:08,EOG,"EOG Resources, Inc.",9,2024-05-11,"EOGResources,Inc.",May,11,2024,"EOGResources,Inc._May_11,_2024",NaN
3,CFRA Equity Research,"Glickman, Stewart",5/11/24 17:57,DVN,Devon Energy Corporation,9,2024-05-11,DevonEnergyCorporation,May,11,2024,"DevonEnergyCorporation_May_11,_2024","Stock Report | May 11, 2024 | NYSESymbol: DVN ..."
4,CFRA Equity Research,"Glickman, Stewart",5/7/24 22:03,COP,ConocoPhillips,9,2024-05-07,ConocoPhillips,May,7,2024,"ConocoPhillips_May_7,_2024",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4887,Argus Research Company,"Selesky, William V.",2/14/19 7:21,PSX,Reaffirming BUY but lowering price target to $116,5,2019-02-14,ReaffirmingBUYbutloweringpricetargetto$116,Feb,14,2019,ReaffirmingBUYbutloweringpricetargetto$116_Feb...,NaN
4888,Argus Research Company,"Selesky, William V.",2/13/19 13:27,SHEL,Reaffirming BUY and target price of $80,5,2019-02-13,ReaffirmingBUYandtargetpriceof$80,Feb,13,2019,"ReaffirmingBUYandtargetpriceof$80_Feb_13,_2019",NaN
4889,Argus Research Company,"Selesky, William V.",2/6/19 7:37,XOM,Strong 4Q18; reaffirming $104 target,5,2019-02-06,Strong4Q18;reaffirming$104target,Feb,6,2019,"Strong4Q18;reaffirming$104target_Feb_6,_2019",NaN
4890,Argus Research Company,"Selesky, William V.",2/5/19 13:39,VLO,Reiterating BUY but lowering price target to $110,5,2019-02-05,ReiteratingBUYbutloweringpricetargetto$110,Feb,5,2019,ReiteratingBUYbutloweringpricetargetto$110_Feb...,NaN


In [22]:
existing_df


,Contributor,Analyst,Date/Time,Company,Headline,Pages,Date,Headline2,Month,Day,Year,Join_Key
0,CFRA Equity Research,"Handshoe, Jonnathan",5/16/24 22:10,MRO,Marathon Oil Corporation,9,2024-05-16,MarathonOilCorporation,May,16,2024,"MarathonOilCorporation_May_16,_2024"
1,CFRA Equity Research,"Glickman, Stewart",5/14/24 22:02,EOG,"EOG Resources, Inc.",9,2024-05-14,"EOGResources,Inc.",May,14,2024,"EOGResources,Inc._May_14,_2024"
2,CFRA Equity Research,"Glickman, Stewart",5/11/24 18:08,EOG,"EOG Resources, Inc.",9,2024-05-11,"EOGResources,Inc.",May,11,2024,"EOGResources,Inc._May_11,_2024"
3,CFRA Equity Research,"Glickman, Stewart",5/11/24 17:57,DVN,Devon Energy Corporation,9,2024-05-11,DevonEnergyCorporation,May,11,2024,"DevonEnergyCorporation_May_11,_2024"
4,CFRA Equity Research,"Glickman, Stewart",5/7/24 22:03,COP,ConocoPhillips,9,2024-05-07,ConocoPhillips,May,7,2024,"ConocoPhillips_May_7,_2024"
...,...,...,...,...,...,...,...,...,...,...,...,...
4887,Argus Research Company,"Selesky, William V.",2/14/19 7:21,PSX,Reaffirming BUY but lowering price target to $116,5,2019-02-14,ReaffirmingBUYbutloweringpricetargetto$116,Feb,14,2019,ReaffirmingBUYbutloweringpricetargetto$116_Feb...
4888,Argus Research Company,"Selesky, William V.",2/13/19 13:27,SHEL,Reaffirming BUY and target price of $80,5,2019-02-13,ReaffirmingBUYandtargetpriceof$80,Feb,13,2019,"ReaffirmingBUYandtargetpriceof$80_Feb_13,_2019"
4889,Argus Research Company,"Selesky, William V.",2/6/19 7:37,XOM,Strong 4Q18; reaffirming $104 target,5,2019-02-06,Strong4Q18;reaffirming$104target,Feb,6,2019,"Strong4Q18;reaffirming$104target_Feb_6,_2019"
4890,Argus Research Company,"Selesky, William V.",2/5/19 13:39,VLO,Reiterating BUY but lowering price target to $110,5,2019-02-05,ReiteratingBUYbutloweringpricetargetto$110,Feb,5,2019,ReiteratingBUYbutloweringpricetargetto$110_Feb...
